In [4]:

import glob
from astropy.io import fits
import numpy as np
from astropy.stats import mad_std
from datetime import date
print ("All libraries imported successfully!") 

All libraries imported successfully!


In [8]:
#This is script must be in the same direcroty as the raw files
#There is a minimum of 3 files needed to clean them from CR.


def clean_cr_from_images( target, mode, ccd_sum):
    
    ####################################
    #1) Cosntants to be used
    ####################################
    
    limit_mad= 10 # a value need to be 'limit_mad' times bigger than the rest of the values to be identified as outlier 
    
    
    
    
    ####################################
    #1) Filter files according to input
    ####################################
    all_files = []
    target_files= []
    
    for file in glob.glob("chi*.fits"):
        fits_image_filename = file[file.find("\\")+1:] 
        
        with fits.open(fits_image_filename) as hdul:

            h_object= hdul[0].header['OBJECT'].strip().lower()
            h_decker= hdul[0].header['DECKER'].strip().lower()
            h_ccd_sum =  hdul[0].header['CCDSUM'].strip().lower()          
            
            if h_object == target and h_decker == mode and h_ccd_sum == ccd_sum:
                #print (" -->> "+h_object + "  "+ h_decker + " " +h_ccd_sum)
                n_columns = int( hdul[0].header['NAXIS1'] )
                n_rows = int( hdul[0].header['NAXIS2'])                
                target_files.append(fits_image_filename)
               
    
    files_num =len(target_files)
    if (files_num <=2 ):
        raise  Exception ("There are not enough file to clean them from CR")
    else:
        print ("Number of files found: " +str (files_num))
        #print (target_files)
    
    #################################################################
    # 2) Create Median Master bias + Master Median Absolute Deviation
    #################################################################
    
    data_cube= np.zeros((n_rows,n_columns,files_num))
    
    for idx in range (files_num):
        with fits.open(target_files[idx]) as hd:
            data = (hd[0].data).copy()            
            data_cube[:,:,idx ] = data

    master_median = np.median(data_cube, axis=2 )
    master_abs_mad = mad_std(data_cube, axis=2 )
    
    #################################################################
    # 2) Cleaning CR from every image + re writing on disk
    #################################################################   
    
    today = date.today() # To be used in all headers
    file_date = today.strftime("%b-%d-%Y")
    
    for f_idx in range (files_num):
        with fits.open(target_files[f_idx]) as hd0:
            #Header
            img_header=(hd0[0].header).copy()
            #cr-cleaned is a keyword used in down the road
            img_header['HISTORY']='CR-CLEANED : Cosmic Rays removed from this file on '+file_date+' by finding the MAD of each pixel in '+str(files_num)+ ' ' +target +' files.'


            #Data 
            img_data=(hd0[0].data).copy()            
            exclude = (img_data - master_median) / master_abs_mad > limit_mad
            img_data[exclude] = 0
            mask = np.multiply( exclude , master_median)
            img_data = np.add(img_data,  mask)    
            
        fits.writeto (target_files[f_idx], img_data, header=img_header, overwrite=True )
        print ("File : "+target_files[f_idx]+  "had "+str(np.count_nonzero(exclude))+"cr. It has been cleaned and overwritten successfully.")

        
        
            
            
        
       
    print(" - - - - -  End of Script - - - - - -  ")
            
    
    




In [6]:
clean_cr_from_images( 'bias','slicer' , '3 1')

Number of files found: 18


UnboundLocalError: local variable 'date' referenced before assignment

In [9]:
clean_cr_from_images( 'sirius','slicer' , '3 1')

Number of files found: 41
File : chi171218.1145.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1146.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1147.fitshad 1cr. It has been cleaned and overwritten successfully.
File : chi171218.1148.fitshad 1cr. It has been cleaned and overwritten successfully.
File : chi171218.1149.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1150.fitshad 1cr. It has been cleaned and overwritten successfully.
File : chi171218.1151.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1152.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1153.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1154.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1155.fitshad 0cr. It has been cleaned and overwritten successfully.
File : chi171218.1156.fitshad 0cr. It h

In [32]:
clean_cr_from_images( 'quartz','slicer' , '3 1')

Number of files found: 30
File : chi171218.1039.fitshad 72cr. It has been cleaned and overwritten successfully.
File : chi171218.1040.fitshad 77cr. It has been cleaned and overwritten successfully.
File : chi171218.1041.fitshad 105cr. It has been cleaned and overwritten successfully.
File : chi171218.1042.fitshad 68cr. It has been cleaned and overwritten successfully.
File : chi171218.1043.fitshad 34cr. It has been cleaned and overwritten successfully.
File : chi171218.1044.fitshad 62cr. It has been cleaned and overwritten successfully.
File : chi171218.1045.fitshad 80cr. It has been cleaned and overwritten successfully.
File : chi171218.1046.fitshad 45cr. It has been cleaned and overwritten successfully.
File : chi171218.1047.fitshad 52cr. It has been cleaned and overwritten successfully.
File : chi171218.1048.fitshad 117cr. It has been cleaned and overwritten successfully.
File : chi171218.1049.fitshad 84cr. It has been cleaned and overwritten successfully.
File : chi171218.1050.fits

In [60]:
# 1) Create Median Master bias + the Master Median Absolute Deviation 


# with fits.open(bias_file) as ref:
#     file=ref[0].data
#     n_columns=file.shape[0]
#     n_rows=file.shape[1]

n_columns=4112
n_rows=1432
images_num = 18

master_median= np.zeros((n_columns,n_rows))
master_abs_mad= np.zeros((n_columns,n_rows))
data_cube= np.zeros((n_columns,n_rows,images_num))




for idx in range (len(bias_files)):
    with fits.open(bias_files[idx]) as hd:
        
        data = hd[0].data
        data_cube[:,:,idx ] = data

        
master_median = np.median(data_cube, axis=2 )
master_abs_mad = mad_std(data_cube, axis=2 )



print ("Master Median + Absolute Median developed ")


# Iterate over each img . For each image 

outFile = 'cr_cleaned/chi_try_1054.fits'
# for j in range (len(bias_files)):

with fits.open(bias_files[0]) as hd0:
    img_header=hd0[0].header
    img_data=hd0[0].data
    
    
    exclude = (img_data - master_median) / master_abs_mad > 10
    img_data[exclude] = 0
   
    
    mask = np.multiply( exclude , master_median)
    
    
    img_data = np.add(img_data,  mask)
    
    print (np.count_nonzero (img_data==0) )
    
    #hd0.writeto(outFile)

hdu=fits.PrimaryHDU(img_data,img_header)
hdul = fits.HDUList([hdu])
hdul.writeto(outFile,overwrite=True)

# n_columns= img_header.shape[0]
# n_row= img_header.shape[1]



#print ( "Number of CR found :" + str(np.sum(exclude))) 
print ("All good !")


        
    
        
        
        

Master Median + Absolute Median developed 
0


C:\Users\mrstu\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\mrstu\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide
C:\Users\mrstu\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in greater


All good !


[ 0 13 14 15 16 17]
